In [135]:
import json
import pandas as pd
  
df = pd.read_json("/Users/oceane/Desktop/plan-your-trip-with-kayak/coordinates.json")

df[['longitude','latitude']] = df.coordinates.apply(lambda x: pd.Series(str(x).split(",")))
df = df.drop(columns=['coordinates', 'city'])


In [136]:
df = df.astype(float)

In [137]:
import requests
weather = []
for index, row in df.iterrows():
    appid = '7e208a67c297443752c742b0cdd73fdf'
    lat= row['latitude']
    lon= row['longitude']
    # URL = 'https://api.openweathermap.org/data/3.0/onecall'
    # PARAMS = {'lat':lat, 'lon':lon, 'appid': appid, 'units': 'metrics', 'cnt':7, 'exclude':'urrent,minutely,hourly,alerts'}
    # r = requests.get(url = URL, params=PARAMS).json()
    URL = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=current,minutely,hourly,alerts&appid={appid}'
    r = requests.get(URL).json()
    for i in range(7):
        weather.append({'lat':lat, 'lon':lon, 'day': r['daily'][i]['dt'], 'temp':r['daily'][i]['temp']['day']})

    

In [138]:

with open('weather.json', 'w') as f:
    f.write(json.dumps(weather, indent=2))


In [139]:
weather_data = pd.read_json("weather.json")
weather_data.head(7)

,lat,lon,day,temp
0,48.635954,-1.51146,1667214000,15.35
1,48.635954,-1.51146,1667300400,16.46
2,48.635954,-1.51146,1667386800,14.21
3,48.635954,-1.51146,1667473200,13.48
4,48.635954,-1.51146,1667559600,12.22
5,48.635954,-1.51146,1667646000,14.22
6,48.635954,-1.51146,1667732400,15.96


In [140]:
weather_data['temp_mean'] = weather_data['temp'].groupby(weather_data['lat']).transform('mean')
weather_data


,lat,lon,day,temp,temp_mean
0,48.635954,-1.511460,1667214000,15.35,14.557143
1,48.635954,-1.511460,1667300400,16.46,14.557143
2,48.635954,-1.511460,1667386800,14.21,14.557143
3,48.635954,-1.511460,1667473200,13.48,14.557143
4,48.635954,-1.511460,1667559600,12.22,14.557143
...,...,...,...,...,...
240,46.159113,-1.152043,1667386800,16.10,16.784286
241,46.159113,-1.152043,1667473200,16.81,16.784286
242,46.159113,-1.152043,1667559600,14.97,16.784286
243,46.159113,-1.152043,1667646000,16.05,16.784286


In [141]:
weather_data = weather_data.drop(columns=['temp'])

In [142]:
weather_data = weather_data.drop_duplicates(subset=['lat', 'lon'])
weather_data = weather_data.reset_index(drop=True)

In [143]:
weather_data

,lat,lon,day,temp_mean
0,48.635954,-1.511460,1667214000,14.557143
1,48.649518,-2.026041,1667214000,14.668571
2,49.276462,-0.702474,1667214000,14.161429
3,49.493898,0.107973,1667214000,14.232857
4,49.440459,1.093966,1667214000,13.681429
5,48.858890,2.320041,1667214000,14.782857
6,49.894171,2.295695,1667214000,13.604286
7,50.636565,3.063528,1667214000,13.121429
8,48.584614,7.750713,1667214000,13.604286
9,48.249490,7.344296,1667214000,10.640000


In [144]:
top_five_weather = weather_data.sort_values(by='temp_mean', ascending=False)

In [145]:
top_five_weather = top_five_weather.head(5)

In [146]:
top_five_weather

,lat,lon,day,temp_mean
27,42.525050,3.083155,1667214000,19.605714
19,43.214036,5.539632,1667214000,18.987143
20,43.296174,5.369953,1667214000,18.820000
18,43.150697,6.341928,1667214000,18.785714
25,43.565823,4.191284,1667214000,18.754286


In [159]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter_mapbox(top_five_weather, lat="lat", lon="lon", color="temp_mean", size="temp_mean", zoom=6, mapbox_style="carto-positron")
fig.show()